# Modelling a simple RNA circuit

In this notebook, we will simulate the interactions between RNAs in a circuit and how the dynamics of the circuit can be simulated.

In [3]:
from src.utils.common.setup_new import construct_circuit_from_cfg
from src.utils.circuit.agnostic_circuits.circuit_manager_new import CircuitModeller

ModuleNotFoundError: No module named 'diffrax'

In [2]:
config = {}
config['data_path'] = 'data/example/toy_mRNA_circuit.fasta'
config = {
    "data_path": "data/example/toy_mRNA_circuit.fasta",
    "experiment": {
        "purpose": "example"
    },
    "molecular_params": "src/utils/common/configs/RNA_circuit/molecular_params.json",
    "system_type": "RNA"
}

circuit = construct_circuit_from_cfg(config_file=config, extra_configs=None)

In [3]:
circuit.__dict__

{'name': 'toy_mRNA_circuit',
 'result_collector': <src.utils.results.results.ResultCollector at 0x7f89fc746fb0>,
 'model': <bioreaction.model.data_containers.BasicModel at 0x7f8998c18a60>,
 'reactions': <bioreaction.model.data_containers.QuantifiedReactions at 0x7f8998c19030>,
 'interactions': <src.srv.parameter_prediction.interactions.MolecularInteractions at 0x7f8998c1ba00>,
 'species_state': 'uninitialised',
 'graph': <src.utils.circuit.agnostic_circuits.circuit_new.Graph at 0x7f89fc7472e0>,
 'circuit_size': 9}